# Foursquare API Examples

In [4]:
import requests
import pandas as pd
import numpy as np
import random
import folium

# converting an address into latitude and longitude values
from geopy.geocoders import Nominatim

# displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# transforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [5]:
# import developer credentials
%run "credentials.py"

In [7]:
# convert address into lat/lng
import certifi
import ssl
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address, timeout=10)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


#### Search for Venues of a Specific Category

In [9]:
# construct URI
LIMIT = 100
search_query = "Italian"
radius = 500
url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [11]:
# send GET request & examine results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1937aa89b06a002c17d95c'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '294544',
     'url': 'https://www.seamless.com/menu/harrys-italian-pizz

In [12]:
# convert json into pandas dataframe
venues = results["response"]["venues"]

df = json_normalize(venues)
df.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",294544,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/harrys-italian-p...,False,4fa862b3e4b0ebff2f749f06,225 Murray St,...,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,Harry's Italian Pizza Bar,v-1561933738
1,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3232e219836c91c7bfde94,63 W Broadway,...,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,Conca Cucina Italian Restaurant,v-1561933738
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520f4e41ee3,124 Chambers St,...,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,Ecco,v-1561933738


In [13]:
df.columns.values

array(['categories', 'delivery.id', 'delivery.provider.icon.name',
       'delivery.provider.icon.prefix', 'delivery.provider.icon.sizes',
       'delivery.provider.name', 'delivery.url', 'hasPerk', 'id',
       'location.address', 'location.cc', 'location.city',
       'location.country', 'location.crossStreet', 'location.distance',
       'location.formattedAddress', 'location.labeledLatLngs',
       'location.lat', 'location.lng', 'location.postalCode',
       'location.state', 'name', 'referralId'], dtype=object)

In [14]:
# select columns of interest
cols = ["name", "categories"] + [col for col in df.columns if col.startswith("location.")] + ["id"]
cols

['name',
 'categories',
 'location.address',
 'location.cc',
 'location.city',
 'location.country',
 'location.crossStreet',
 'location.distance',
 'location.formattedAddress',
 'location.labeledLatLngs',
 'location.lat',
 'location.lng',
 'location.postalCode',
 'location.state',
 'id']

In [15]:
# filter dataframe
df_filtered = df.loc[:, cols]
df_filtered

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,id
0,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [16]:
# extract category of each venue
def get_category(row):
    try:
        categories_list = row["categories"]
    except:
        categories_list = row["venue.categories"]
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]["name"]
    
df_filtered["categories"] = df_filtered.apply(get_category, axis=1)
df_filtered

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [17]:
# clean up column names
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]
df_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [18]:
# visualize venues
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.vector_layers.CircleMarker(
    [latitude, longitude],
    radius=5,
    color="red",
    fill=True,
    fill_color="red",
    fill_opacity=0.6
).add_to(venues_map)


for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.name):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=3,
        color="blue",
        fill=True,
        fill_color="red",
        fill_opacity=0.6,
        popup=label
    ).add_to(venues_map)

venues_map

#### Explore a Specific Venue

In [19]:
venue_id = "3fd66200f964a520f4e41ee3"
url = "https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}".format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

In [20]:
result = requests.get(url).json()
venue = result["response"]["venue"]

In [21]:
name = venue["name"]
num_tips = venue["tips"]["count"]
try:
    rating = venue["rating"]
except:
    rating = "None"
print("{} has a rating of {} and has {} tips".format(name, rating, num_tips))

Ecco has a rating of 8.0 and has 17 tips


#### Get Venue Tips

In [22]:
limit = 15
url = "https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}".format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)
results = requests.get(url).json()

In [23]:
tips = results["response"]["tips"]["items"]
tips

[{'id': '5ab1cb46c9a517174651d3fe',
  'createdAt': 1521601350,
  'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
  'type': 'user',
  'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
  'lang': 'en',
  'likes': {'count': 0, 'groups': []},
  'logView': True,
  'agreeCount': 3,
  'disagreeCount': 0,
  'lastVoteText': 'Upvoted May 14',
  'lastUpvoteTimestamp': 1557868336,
  'todo': {'count': 0},
  'user': {'id': '484542633',
   'firstName': 'Nick',
   'lastName': 'E',
   'gender': 'male',
   'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
    'suffix': '/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'}},
  'authorInteractionType': 'liked'}]

In [24]:
pd.set_option("display.max_colwidth", -1)

tips_df = json_normalize(tips)
cols = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']

tips_filtered = tips_df.loc[:, cols]
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",3,0,5ab1cb46c9a517174651d3fe,Nick,E,male,484542633


#### Search for a User

In [25]:
user_id = "484542633"
url = "https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}".format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION)

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1937bcdbde1100250fddb8'},
 'notifications': [],
 'response': {'user': {'id': '484542633',
   'firstName': 'Nick',
   'lastName': 'E',
   'gender': 'male',
   'canonicalUrl': 'https://foursquare.com/nickeltawil',
   'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
    'suffix': '/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'},
   'friends': {'count': 0,
    'groups': [{'type': 'others',
      'name': 'Other friends',
      'count': 0,
      'items': []}]},
   'tips': {'count': 239},
   'homeCity': 'New York, NY',
   'bio': 'https://www.tawil.team/nick-el-tawil/',
   'contact': {},
   'photos': {'count': 0, 'items': []},
   'type': 'user',
   'mayorships': {'count': 0, 'items': []},
   'checkins': {'count': 1, 'items': []},
   'lists': {'count': 2,
    'groups': [{'type': 'created', 'count': 0, 'items': []},
     {'type': 'followed', 'count': 0, 'items': []},
     {'type': 'yours',
      'count': 2,

In [27]:
user_data = results["response"]["user"]
first_name = user_data["firstName"]
photo = user_data["photo"]
num_tips = user_data["tips"]["count"]

print("{} has published {} tips".format(first_name, num_tips))
Image(url="https://igx.4sqi.net/img/user/300x300"+photo["suffix"])

Nick has published 239 tips


#### Get a User's Tips

In [28]:
url = "https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}".format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
tips = results["response"]["tips"]["items"]

pd.set_option("display.max_colwidth", -1)

tips_df = json_normalize(tips)

filtered_columns = ["text", "agreeCount", "disagreeCount", "id"]
tips_filtered = tips_df.loc[:, filtered_columns]
tips_filtered

,text,agreeCount,disagreeCount,id
0,"The best! I’m especially fond of the salmon burger, but I’ve had half of the menu and never been disappointed. There’s a reason this place is well known even outside of the Village!",1,0,5aec594b1f7440002c138612
1,I used to down a pint of chocolate like it was nothing back when I was bulking. Highly recommended!,1,0,5accc9f66fa81f196724807b
2,They serve coffee!!!!!!,1,0,5accc98c0313204c9d7ec157
3,"I’m a fan. In fact, I’m such a big fan, I want Taim to hire me to be their spokesman. Kind of like the Arabic Jared Fogle - but without the kid stuff.",1,0,5accbf033abcaf09a24612a0
4,The linguine with clams is on point 👌,1,0,5accbe3a911fc423730f3ed3
5,"Great for a quick, cheap lunch! Shorter lines than Chipotle too👌",1,0,5acbecb86fa81f1967e019b0
6,"Quick, cheap lunch that tastes good! Way shorter line than Chipotle, too.",1,0,5acbec70a0215b732e264fe8
7,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
8,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,2,0,5acbbcda01235808d5d6dc75
9,Coffee game on point,1,0,5acbbb1501235808d5d6525e


#### Search for a Tip

In [29]:
tip_id = "5ab5575d73fe2516ad8f363b"

url = "http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}".format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
print(result["response"]["tip"]["venue"]["name"])

Cowgirl


#### Explore Nearby Locations

In [30]:
latitude = 40.715337
longitude = -74.008848

url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [31]:
results = requests.get(url).json()
items = results["response"]["groups"][0]["items"]
print("Found {} venues.".format(len(items)))

Found 100 venues.


In [32]:
items_df = json_normalize(items)
filtered_columns = ["venue.name", "venue.categories"] + [col for col in items_df.columns if col.startswith("venue.location.")] + ["venue.id"]
items_filtered = items_df.loc[:,filtered_columns]

items_filtered.columns = [col.split(".")[-1] for col in items_filtered.columns]
items_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Korin,"[{'id': '4bf58dd8d48988d1f8941735', 'name': 'Furniture / Home Store', 'pluralName': 'Furniture / Home Stores', 'shortName': 'Furniture / Home', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/furniture_', 'suffix': '.png'}, 'primary': True}]",57 Warren St,US,New York,United States,Church St,73,"[57 Warren St (Church St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}]",40.714824,-74.009404,Tribeca,10007,NY,4af5d65ff964a52091fd21e3
1,Juice Press,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'Vegetarian / Vegan Restaurant', 'pluralName': 'Vegetarian / Vegan Restaurants', 'shortName': 'Vegetarian / Vegan', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_', 'suffix': '.png'}, 'primary': True}]",83 Murray St,US,New York,United States,btwn Greenwich St & W Broadway,202,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",40.714788,-74.011132,NaN,10007,NY,54148bc6498ea7bb8c05b70a
2,Takahachi Bakery,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'Bakery', 'pluralName': 'Bakeries', 'shortName': 'Bakery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_', 'suffix': '.png'}, 'primary': True}]",25 Murray St,US,New York,United States,at Church St,187,"[25 Murray St (at Church St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}]",40.713653,-74.008804,NaN,10007,NY,4c154c9a77cea593c401d260
3,Takahachi,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'Sushi Restaurant', 'pluralName': 'Sushi Restaurants', 'shortName': 'Sushi', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_', 'suffix': '.png'}, 'primary': True}]",145 Duane St,US,New York,United States,btwn W Broadway & Church St,146,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}]",40.716526,-74.008101,NaN,10013,NY,4a8f2f39f964a520471420e3
4,Chambers Street Wines,"[{'id': '4bf58dd8d48988d119951735', 'name': 'Wine Shop', 'pluralName': 'Wine Shops', 'shortName': 'Wine Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_wineshop_', 'suffix': '.png'}, 'primary': True}]",148 Chambers St,US,New York,United States,btwn West Broadway & Hudson St,88,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}]",40.715773,-74.009718,NaN,10007,NY,4adcf23cf964a520cc6221e3


In [33]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15)

folium.vector_layers.CircleMarker(
    [latitude, longitude],
    radius=5,
    color="red",
    fill=True,
    fill_color="red",
    fill_opacity=0.6
).add_to(venues_map)

for lat, lng, label in zip(items_filtered.lat, items_filtered.lng, items_filtered.name):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=3,
        color="blue",
        fill=True,
        fill_color="red",
        fill_opacity=0.6,
        popup=label
    ).add_to(venues_map)
venues_map

#### Explore Trending Venues

In [34]:
# check whether any venues are trending (run time dependent)
def get_trending_venues(latitude, longitude):
    url = "https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}".format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)
    results = requests.get(url).json()
    trending = results["response"]["venues"]
    
    if len(trending) == 0:
        return "No venues are trending at this time."
    else:
        trending_df = json_normalize(trending)
        
        # Re-order and filter columns
        columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
        trending_filtered = trending_df.loc[:, columns_filtered]
        
        # Rename columns
        trending_filtered.columns = [col.split(".")[-1] for col in trending_filtered.columns]
        
        # Get category name
        trending_filtered["category"] = trending_filtered.apply(get_category, axis=1)
        
        return trending_filtered
        

In [35]:
trending_venues = get_trending_venues(latitude, longitude)
trending_venues

,name,categories,distance,city,postalCode,state,country,lat,lng,category
0,Washington Square Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}]",1972,New York,10012,NY,United States,40.730816,-73.997458,Park
1,NYC Pride March,"[{'id': '52741d85e4b0d5d1e3c6a6d9', 'name': 'Parade', 'pluralName': 'Parades', 'shortName': 'Parade', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/event/default_', 'suffix': '.png'}, 'primary': True}]",1929,New York,10014,NY,United States,40.732657,-74.008112,Parade
2,Morgenstern’s Finest Ice Cream,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'Ice Cream Shop', 'pluralName': 'Ice Cream Shops', 'shortName': 'Ice Cream', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_', 'suffix': '.png'}, 'primary': True}]",1508,New York,10012,NY,United States,40.727057,-73.999864,Ice Cream Shop
3,One World Trade Center,"[{'id': '4bf58dd8d48988d130941735', 'name': 'Building', 'pluralName': 'Buildings', 'shortName': 'Building', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_', 'suffix': '.png'}, 'primary': True}]",440,New York,10007,NY,United States,40.713069,-74.013133,Building


In [36]:
def plot_venues(latitude, longitude, venues):
    if not isinstance(venues, pd.DataFrame):
        return "Cannot create map as there are no trending venues at this time."
    else:
        venues_map = folium.Map(location=[latitude, longitude], zoom_start=15)

        folium.vector_layers.CircleMarker(
            [latitude, longitude],
            radius=5,
            color="red",
            fill=True,
            fill_color="red",
            fill_opacity=0.6
        ).add_to(venues_map)

        for lat, lng, label in zip(venues.lat, venues.lng, venues.name):
            folium.vector_layers.CircleMarker(
                [lat, lng],
                radius=3,
                color="blue",
                fill=True,
                fill_color="red",
                fill_opacity=0.6,
                popup=label
            ).add_to(venues_map)
        return venues_map

In [37]:
venues_map = plot_venues(latitude, longitude, trending_venues)
venues_map